<a target="_blank" href="https://colab.research.google.com/github/farahshamout/AIP-2024/blob/main/Week%204/%5BDay%202%2C%20Task%202%5D%20Chatbot%20Architectures.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Retrieval Augmented Generation and Chatbot Architectures

In this notebook we will work through what it takes to build a chatbot from the most basic version, all the way through to a more complicated chatbot that uses a conversational agent with tools as well as guardrails.

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk-..."  #put the API key in here

In [ ]:
# %pip install --upgrade gradio
# %pip install langchain
# %pip install langsmith
# %pip install langchain-community
# %pip install openai
# %pip install -U langchain-openai

## The most basic Chatbot possible:

This simply uses a user interface attached to a LLM

In [ ]:
from langchain_openai import ChatOpenAI
import gradio as gr

llm = ChatOpenAI(temperature=1.0, model='gpt-3.5-turbo-16k')

def predict(message, history):
    gpt_response = llm.invoke(message)
    return gpt_response.content

gr.ChatInterface(predict).launch()

## Retrieval Augmented Generation Example

### Source and clean data for RAG

In [ ]:
# %pip install bs4

In [ ]:
import requests
from bs4 import BeautifulSoup
import re

def scrape_website_text(url):
    # Send a GET request to the Wikipedia page
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content of the page
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extract all the text from paragraphs and other relevant tags
        paragraphs = soup.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6'])
        text = ' '.join([para.get_text() for para in paragraphs])
        
        # Clean the text by removing special characters
        cleaned_text = re.sub(r'[^A-Za-z0-9\s]', '', text)
        
        return cleaned_text.strip()
    else:
        return "Failed to retrieve the page"

In [ ]:
url = """https://www.cbsnews.com/news/paris-olympics-2024-200-meters-noah-lyles-kenny-bednarek-letsile-tebogo/"""  # put any website page in here
cleaned_text = scrape_website_text(url)

# you can test that this has worked using the line below (uncomment to use)
# print(cleaned_text)

### Split and chunk the text for embedding

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)

In [ ]:
texts = text_splitter.create_documents([cleaned_text])
print(texts[0])
print(texts[1])

### Embed the documents and save in a Vector database

In [ ]:
# %pip install faiss-cpu

The Cell below does cost money to run, however the embeddings don't have to be done every time. Once a vector database is made, it can be saved and retrieved for later use.

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")

vector_database = FAISS.from_documents(texts, embedding_model)

Below is a test to see if we can retrieve some relevant context chunks from our document store:

In [ ]:
query = "Who won gold in the men's 200m sprint at the 2024 Paris Olympics?"

print(vector_database.similarity_search(query, 3))

### Include the RAG pipeline into our chatbot prototype:

In [ ]:
from langchain_openai import ChatOpenAI
import gradio as gr

llm = ChatOpenAI(temperature=1.0, model='gpt-3.5-turbo-16k')

def predict(message, history):
    context = vector_database.similarity_search(message, 3)
    formatted_prompt = f"""{context} \nUse the above context to answer the follwing question: \n{message}"""
    gpt_response = llm.invoke(formatted_prompt)
    return gpt_response.content

gr.ChatInterface(predict).launch()

## RAG Chatbot with Conversational History

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage
import gradio as gr

llm = ChatOpenAI(temperature=1.0, model='gpt-3.5-turbo-16k')

def predict(message, history):
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    history_langchain_format.append(HumanMessage(content=message))
    context = vector_database.similarity_search(message, 3)
    formatted_prompt = f"""{context} \nUse the above context to answer the follwing question: \nHere is your conversation history with the user and their latest question: {history_langchain_format}"""
    print(history_langchain_format)
    print(context)
    gpt_response = llm.invoke(formatted_prompt)
    return gpt_response.content

gr.ChatInterface(predict).launch()

## Conversational Agent

### import the libraries

In [ ]:
from langchain.agents import Tool
from langchain.tools import tool
from langchain.globals import set_debug
set_debug(True)

In [ ]:
# %pip install wikipedia
# %pip install langchainhub

### Define the tools

In [ ]:
# defining the tool for the context retrieval function
@tool
def context_retreival(query: str) -> str:
    """This tool returns relevant context documents about the 200m sprint at the 2024 olympics in Paris. Ask this tool any question about the 2024 200m sprint finals and who won it."""
    
    context = vector_database.similarity_search(query, 3)

    return context

In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# defining the tool for the wiki retrieval function
@tool
def wiki_lookup(query: str) -> str:
    """This tool should be used for questions about particular people and their achievements. It returns a context document from wikipedia."""
    
    wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
    
    context = wikipedia.run(query)

    return context

In [ ]:
tools = [
    context_retreival,
    wiki_lookup
]

### Initialise the prompt and agent

In [ ]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

In [ ]:
from langchain.agents import create_tool_calling_agent
agent = create_tool_calling_agent(llm, tools, prompt)

In [ ]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

### Launch a user interface for experimentation:

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage
import gradio as gr

llm = ChatOpenAI(temperature=1.0, model='gpt-3.5-turbo-16k')

def predict(message, history):
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    history_langchain_format.append(HumanMessage(content=message))

    gpt_response = agent_executor.invoke({"input": message, "chat_history": history_langchain_format})
    return gpt_response['output']


gr.ChatInterface(predict).launch(debug=True)